## Dataset
1. https://github.com/zxaoyou/segmentation_WBC
2. https://raabindata.com/free-data/

In [10]:
import csv
import sys

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from keras import regularizers, optimizers
import os
import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import shutil
import logging

In [33]:
# Configuration
force = True # Force the image preprocessing again
types = ['Basophil', 'Eosinophil', 'Lymphocyte', 'Monocyte', 'Neutrophil']
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)

raw_train = 'data/raw/train/'
raw_test = 'data/raw/test/'

train_folder = 'data/train/'
validation_folder = 'data/validation/'
test_folder = 'data/test/'

# percentage of each cell present in each dataset

train_size = 0.70
validation_size = 0.15
test_size = 0.15



In [34]:
def create_generator(folder, subset):
    df = pd.read_csv(folder + '/class.csv', dtype=str, delimiter=',')
    data = ImageDataGenerator(rescale=1./255.)
    return data.flow_from_dataframe(
        dataframe=df,
        directory=folder,
        x_col='Image',
        y_col='Id',
        subset=subset,
        batch_size=32,
        shuffle=True,
        class_mode='categorical',
        target_size=(120, 120),
        color_mode='grayscale',
    )

def gray_scale(image, dest):
    im = Image.open(image)
    p = np.array(im)
    # p = np.dot(p[...,:3], [0.2999, 0.587, 0.114])
    p = p[:, :, 0] > 125
    # visual = np.log(p)
    result = Image.fromarray(p.astype(np.uint8))
    result.save(dest)

In [36]:
logging.info("Starting program...")
classification = None

if not os.path.isdir('data/train') or force is True:

    logging.warning("No train folder detected...")
    logging.info("Creating folders...")

    if not os.path.isdir(train_folder):
        os.mkdir(train_folder)
        os.mkdir(test_folder)

    # Load Train
    classification = open(train_folder, 'w', newline='')
    writer = csv.writer(classification)
    writer.writerow(['Id', 'Image'])
    i = 1
    for c in types:
        for f in os.listdir(raw_train + c):
            # if os.path.isfile('data/' + c):
            shutil.copy(raw_train + c + '/' + f, train_folder + '/' + f)
            writer.writerow([train_folder + '/' + f, i])


"""
    with open(dataset_folder + '.csv', 'r') as f:
        reader = csv.reader(f)
        for line in reader:
            classification.append(line[1])
    if not os.path.isdir('data/train'):
        os.mkdir(train_folder)
        os.mkdir(validation_folder)
        os.mkdir(test_folder)

    i = 1
    file = None
    writer = None
    current = train_folder
    for f in os.listdir(dataset_folder):
        if i == 1:
            logging.info("Creating test dataset...")
            file = open(train_folder + '/class.csv', 'w', newline='')
            writer = csv.writer(file)
            writer.writerow(['Image', 'Id'])
        elif i == train_size :
            logging.info("Creating validation dataset...")
            file.close()
            file = open(validation_folder + '/class.csv', 'w', newline='')
            current = validation_folder
            writer = csv.writer(file)
            writer.writerow(['Image', 'Id'])
        elif i == train_size + validation_size:
            logging.info("Creating test dataset...")
            file.close()
            file = open(test_folder + '/class.csv', 'w', newline='')
            current = test_folder
            writer = csv.writer(file)
            writer.writerow(['Image', 'Id'])

        if os.path.isfile(dataset_folder + '/' + f) and 'bmp' in f:
            gray_scale(dataset_folder + '/' + f, current + '/' + f)
            writer.writerow([f, classification[i]])
            i += 1
    file.close()
    logging.info("Creation complete!")
    print(classification)
else:
    logging.info("Train folder detected!")
"""
logging.info("Starting WBC classification...")

INFO:Starting program...
INFO:Creating folders...


PermissionError: [Errno 13] Permission denied: 'data/train/'

In [35]:
logging.info("Creating generator...")
train_generator = create_generator(train_folder, 'training')
validation_generator = create_generator(validation_folder, 'validation')
test_generator = create_generator(test_folder, None)

INFO:Creating generator...


Found 139 validated image filenames belonging to 4 classes.
Found 0 validated image filenames belonging to 3 classes.
Found 131 validated image filenames belonging to 5 classes.


In [36]:
logging.info("Training...")
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(120, 120, 1)))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_generator, epochs=50)

INFO:Training...


Epoch 1/50
5/5 [==============================] - 1s 161ms/step - loss: 1.3218 - accuracy: 0.2806
Epoch 2/50
5/5 [==============================] - 1s 114ms/step - loss: 1.2370 - accuracy: 0.3885
Epoch 3/50
5/5 [

In [37]:
for e in classification:
    if e == '5':
        print(e)

print(test_generator.class_indices)

5
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4}


In [38]:
score = model.evaluate(test_generator)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

InvalidArgumentError:  logits and labels must be broadcastable: logits_size=[32,4] labels_size=[32,5]
	 [[node categorical_crossentropy/softmax_cross_entropy_with_logits
 (defined at C:\Users\Paul\Projects\Python\WBC\venv\lib\site-packages\keras\backend.py:5009)
]] [Op:__inference_test_function_13154]

Errors may have originated from an input operation.
Input Source operations connected to node categorical_crossentropy/softmax_cross_entropy_with_logits:
In[0] categorical_crossentropy/softmax_cross_entropy_with_logits/Reshape:	
In[1] categorical_crossentropy/softmax_cross_entropy_with_logits/Reshape_1:

Operation defined at: (most recent call last)
>>>   File "C:\Users\Paul\AppData\Local\Programs\Python\Python38\lib\runpy.py", line 194, in _run_module_as_main
>>>     return _run_code(code, main_globals, None,
>>> 
>>>   File "C:\Users\Paul\AppData\Local\Programs\Python\Python38\lib\runpy.py", line 87, in _run_code
>>>     exec(code, run_globals)
>>> 
>>>   File "C:\Users\Paul\Projects\P

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())